In [ ]:
# /**********************************************************************************************************

# File Name		: 	bigqueryML.ipynb
# Purpose			:   Applying Kmeans model with Bigquery ML for integrated data
# Author			:   DeepSphere.AI, Inc.
# Date and Time 	: 	03/16/2021 10:30 hrs
# Version			: 	1.0 

# /************************************************************************************************************


def kmeans_model():
    vAR_model_flag = create_model()
    if vAR_model_flag:
        print('Model created successfully')
    else:
        print('Error in model creation')
    vAR_predict_flag = model_predict()
    if vAR_model_flag:
        print('Predicted result successfully saved')
    else:
        print('Error in prediction')
    return True

In [ ]:
def create_model():
    from google.cloud import bigquery
    # Construct a BigQuery client object.
    vAR_client = bigquery.Client()

    # TODO(developer): Set table_id to the ID of the destination table.
    vAR_sql = """
        CREATE OR REPLACE MODEL
      ds_ai.ds_ai_kmeans_model OPTIONS(model_type='kmeans',
        num_clusters=3) AS
      SELECT * EXCEPT(uuid,customer_id) from `vast-verve-292018.ds_ai.ds_ai_integrated_data`
    """
    # Start the query, passing in the extra configuration.
    query_job = vAR_client.query(vAR_sql)  # Make an API request.
    query_job.result()  # Wait for the job to complete.
    return True

In [ ]:
def model_predict():
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    vAR_client = bigquery.Client()
    # TODO(developer): Set table_id to the ID of the destination table.
    vAR_table_id = "vast-verve-292018.ds_ai.ds_ai_kmeans_op"

    vAR_job_config = bigquery.QueryJobConfig(destination=vAR_table_id,write_disposition="WRITE_TRUNCATE")

    vAR_sql = """SELECT * EXCEPT(nearest_centroids_distance,uuid,summary_or_insights,text_content)
    FROM
      ML.PREDICT( MODEL ds_ai.ds_ai_kmeans_model,
        (SELECT * FROM ds_ai.ds_ai_integrated_data))
    """
    # Start the query, passing in the extra configuration.
    vAR_query_job = vAR_client.query(vAR_sql, job_config=vAR_job_config)  # Make an API request.
    vAR_query_job.result()  # Wait for the job to complete.

    print("Predicted results loaded to the table {}".format(vAR_table_id))
    return True

In [ ]:
# /**********************************************************************************************************
# Disclaimer.

# We are providing this code block strictly for learning and researching, this is not a production ready code.
# We have no liability on this particular code under any circumstances; users should
# use this code on their own risk. All software,hardware and other products that are
# referenced in these materials belong to the respective vendor who developed or who owns 
# this product.
# /**********************************************************************************************************
